In [ ]:
import os
import json
import argparse
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
)


In [ ]:
class ProductDataset(Dataset):
    def __init__(self, df, tokenizer, label_mapping=None, max_length=256, is_train=True):
        """
        :param df: DataFrame с данными
        :param tokenizer: токенизатор Hugging Face
        :param label_mapping: словарь mapping оригинальных cat_id -> числовой индекс (только для обучающего набора)
        :param max_length: максимальная длина последовательности
        :param is_train: True, если набор данных используется для обучения (с метками)
        """
        self.df = df
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.is_train = is_train
        self.label_mapping = label_mapping

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        row = self.df.iloc[index]
        text = row['text']
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors="pt"
        )
        # Убираем лишнее измерение
        encoding = {key: val.squeeze() for key, val in encoding.items()}
        if self.is_train:
            # Преобразуем оригинальный cat_id в числовую метку с помощью label_mapping
            orig_label = str(row['label'])
            if self.label_mapping is not None:
                label = self.label_mapping.get(orig_label, -1)
            else:
                label = int(row['label'])
            encoding['labels'] = torch.tensor(label, dtype=torch.long)
        return encoding

In [ ]:
import torch.nn as nn
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds),
    }
def train_model(df,val_df):
    """
    Обучает модель на размеченных данных из файла labeled_train.parquet.
    Сохраняет модель, токенизатор и маппинг меток в директорию ./model.
    """

    os.environ["WANDB_DISABLED"] = "true"



    unique_labels = sorted(df['label'].unique())
    label2id = {str(cat): idx for idx, cat in enumerate(unique_labels)}
    id2label = {idx: str(cat) for idx, cat in enumerate(unique_labels)}
    

    model_name = "answerdotai/ModernBERT-base"
    
    tokenizer = AutoTokenizer.from_pretrained(model_name,use_fast=True)
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name,
        num_labels=len(unique_labels),
        id2label=id2label,
        label2id=label2id
    )



    train_dataset = ProductDataset(df, tokenizer, label_mapping=label2id, max_length=512, is_train=True)
    val_dataset = ProductDataset(val_df, tokenizer, label_mapping=label2id, max_length=512, is_train=True)
    data_collator = DataCollatorWithPadding(tokenizer)

    training_args = TrainingArguments(
        output_dir="./model1",
        num_train_epochs=1,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        eval_strategy='steps',
        eval_steps=250,
        #save_strategy="epoch",
        logging_dir='./logs',
        logging_steps=500,
        learning_rate=5e-5,
        #save_only_model=1,
        optim="adamw_torch_fused", # improved optimizer 
        logging_strategy="steps",
        report_to=None  # отключаем логирование в WandB и т.п.
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
        data_collator=data_collator
    )
    
    trainer.train()
    
    # Сохраняем модель и токенизатор
    trainer.save_model("./model1")
    tokenizer.save_pretrained("./model1")
    
    # Сохраняем mapping меток для последующего использования
    mapping = {"label2id": label2id, "id2label": id2label}
    with open(os.path.join("model1", "label_mapping.json"), "w") as f:
        json.dump(mapping, f)
    
    print("Модель обучена и сохранена в директорию ./model")
    return model, tokenizer, label2id, id2label, trainer

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
df = pd.read_csv("/kaggle/input/datafacts/train.csv")
def preprocess_text(text):
    if pd.isna(text):
        return ""
    return str(text).strip()

df['text'] = df['text'].apply(preprocess_text)
df['label'] = df['label'].astype(int)

df = df[df['text'].str.len() > 0]
train_df, val_df = train_test_split(
    df, 
    test_size=0.2, 
    random_state=42,  
)



In [ ]:
model, tokenizer, l2id, i2l, train_result = train_model(train_df, val_df)

In [ ]:
metrics = train_result.metrics
trainer.save_metrics("train", metrics)

# Оценка на валидационном наборе
val_metrics = trainer.evaluate()
trainer.save_metrics("eval", val_metrics)

print("\nРезультаты обучения:")
print(f"Тренировочные потери: {metrics['train_loss']:.4f}")
print(f"Валидационная точность: {val_metrics['eval_accuracy']:.4f}")
print(f"Валидационный F1: {val_metrics['eval_f1']:.4f}")
